# IPM DSS

## import modules

In [1]:
import pandas

from openalea.dss import Hub, Model

from collections import UserDict

## Access to DSS catalog

In [251]:
#hub = Hub()
#hub.list_dss(ViewDataFrame=True)
#print(hub) 

#hub.display(ViewDataFrame=False) #to improve
import pandas
from agroservices import IPM

class Hub:
    """[summary]
    """
    
    def __init__(self):
        """[summary]
        """
        self.ipm_hub=IPM()
        self._catalog = None
    
    @property 
    def catalog(self):
        """[summary]

        Returns
        -------
        [type]
            [description]
        """
        if self._catalog is None:
            self._catalog = self.ipm_hub.get_dss()
            
        return  {el["id"]: {item["id"]:item for item in el["models"]} for el in self._catalog}
    
    
    def display(self,view="dataframe"):
        if view=="dataframe":
            df=pandas.Series(self.catalog).apply(pandas.Series).stack().apply(pandas.Series)
            df=df[["pests","crops","description","input","output"]]
            df=pandas.concat([df.drop("description",axis=1),df['description'].apply(pandas.Series)],axis=1)
            df.rename(columns={'other':'description'}, inplace=True)
            df=df.drop(['created_by', 'age', 'assumptions', 'peer_review', 'case_studies'],axis=1)
            
            return df
        
        else:
            print(self.catalog)
    
    @ property
    def list_dss(self):
        return list(self.catalog)
    

    @property    
    def nb_model(self):
        return len(self.catalog)

In [252]:
h=Hub()
h.display()

pests  \
no.nibio.vips         PSILARTEMP                                                       [PSILRO]   
                      DELIARADIC                                                       [HYLERA]   
                      MAMESTRABR                                                       [BARABR]   
                      PSILAROBSE                                                       [PSILRO]   
                      DELIARFOBS                                               [HYLERA, HYLEFL]   
                      NAERSTADMO                                                       [PHYTIN]   
                      ALTERNARIA                                                       [ALTESO]   
                      NEGPROGMOD                                                       [PHYTIN]   
                      SEPAPIICOL                                                       [SEPTAP]   
                      BREMIALACT                                                       [BREMLA]   
adas.datamanipulation CIBSEsingleday                                                       None   
                      CIBSEmultipledays                                                    None   
                      Sin14R-1singleday                                                    None   
                      Sin14R-1multipledays                                                 None   
                      Hourly_RH                                                            None   
                      LeafWetnessDuration_RH                                               None   
                      LeafWetnessDuration                                                  None   
gr.gaiasense.ipm      PLASVI                                                           [PLASVI]   
adas.dss              DASGPA                                                           [DASGPA]   
                      SITDMO                                                           [SITDMO]   
                      MELIAE                                                           [MELIAE]   
                      HAPDMA                                                           [HAPDMA]   
                      CARPPO                                                           [CARPPO]   
                      RHOPPA                                                           [RHOPPA]   
com.ipmwise           ipmwise.dk              [ACHMI, AETCY, AGSST, ALOGE, ALOMY, AMSCA, ANG...   
                      ipmwise.no              [ANGAR, CONCA, SENVU, SENJA, SENVE, BIDTR, GAL...   
                      ipmwise.es              [ANGAR, CONCA, SENVU, SENJA, SENVE, BIDTR, GAL...   
                      ipmwise.demo            [ACHMI, AETCY, AGSST, ALOGE, ALOMY, AMSCA, ANG...   
dk.seges              SEPTORIAHU                                       [SEPTTR, LEPTNO, PYRNTR]   
                      CPO                     [SEPTTR, PUCCST, PUCCRE, ERYSGR, PSDCHE, PYRNT...   

                                                                                          crops  \
no.nibio.vips         PSILARTEMP                                                        [DAUCS]   
                      DELIARADIC                                                        [1BRSG]   
                      MAMESTRABR                                                        [1BRSG]   
                      PSILAROBSE                                                        [DAUCS]   
                      DELIARFOBS                                          [BRSOL, BRSOB, BRSOK]   
                      NAERSTADMO                                                        [SOLTU]   
                      ALTERNARIA                                                        [SOLTU]   
                      NEGPROGMOD                                                        [SOLTU]   
                      SEPAPIICOL                                                        [APUGD]   
                      BREMIALA

In [247]:
h.catalog

{'no.nibio.vips': {'PSILARTEMP': {'name': 'Carrot rust fly temperature model',
   'id': 'PSILARTEMP',
   'version': '1.0',
   'type_of_decision': 'Short-term tactical',
   'type_of_output': 'Risk indication',
   'description_URL': 'https://www.vips-landbruk.no/forecasts/models/PSILARTEMP/',
   'citation': None,
   'keywords': 'none',
   'platform_validated': True,
   'pests': ['PSILRO'],
   'crops': ['DAUCS'],
   'authors': [{'name': 'Berit Nordskog',
     'email': 'berit.nordskog@nibio.no',
     'organization': 'NIBIO'}],
   'execution': {'type': 'ONTHEFLY',
    'endpoint': 'https://coremanager.vips.nibio.no/models/PSILARTEMP/run/ipmd',
    'form_method': 'post',
    'content_type': 'application/json',
    'input_schema': '{"type":"object","properties":{"modelId":{"type":"string","pattern":"^PSILARTEMP$","title":"Model Id","default":"PSILARTEMP","description":"Must be PSILARTEMP"},"configParameters":{"title":"Configuration parameters","type":"object","properties":{"timeZone":{"type":"

In [182]:
df=pandas.Series(h.dss_catalog).apply(pandas.Series).stack().apply(pandas.Series)

df=df[['name','pests', 'crops','platform_validated','description']]
df=pandas.concat([df.drop("description",axis=1),df['description'].apply(pandas.Series)],axis=1)
df.rename(columns={'other':'description'}, inplace=True)
df=df.drop(['created_by', 'age', 'assumptions', 'peer_review', 'case_studies'],axis=1)
df

name  \
no.nibio.vips         PSILARTEMP                              Carrot rust fly temperature model   
                      DELIARADIC                    Cabbage fly flight period temperature model   
                      MAMESTRABR                                             Cabbage moth model   
                      PSILAROBSE                              Carrot rust fly observation model   
                      DELIARFOBS              Cabbage root fly and turnip fly observation model   
                      NAERSTADMO                                                  Nærstad model   
                      ALTERNARIA                                             Alternaria TOMCAST   
                      NEGPROGMOD                                             Negative prognosis   
                      SEPAPIICOL                                        Septoria apiicola model   
                      BREMIALACT                Bremia lactucae (Downy mildew of lettuce) model   
adas.datamanipulation CIBSEsingleday                  CIBSE hourly temperature for a single day   
                      CIBSEmultipledays              CIBSE hourly temperature for multiple days   
                      Sin14R-1singleday            Sin14R-1 hourly temperature for a single day   
                      Sin14R-1multipledays        Sin14R-1 hourly temperature for multiple days   
                      Hourly_RH                                         Hourly Relative Humidiy   
                      LeafWetnessDuration_RH                           Leaf Wetness Duration_RH   
                      LeafWetnessDuration                                 Leaf Wetness Duration   
gr.gaiasense.ipm      PLASVI                                          Downy mildew of grapevine   
adas.dss              DASGPA                                                      Cutworm Model   
                      SITDMO                                   Orange Wheat Blossom Midge Model   
                      MELIAE                                                Pollen Beetle Model   
                      HAPDMA                                            Saddle Gall Midge Model   
                      CARPPO                                                 Codling Moth Model   
                      RHOPPA                                                         TSUM Model   
com.ipmwise           ipmwise.dk                                                IPMwise Denmark   
                      ipmwise.no                                              VIPS-ugras Norway   
                      ipmwise.es                                                  IPMwise Spain   
                      ipmwise.demo                                         IPMwise demo version   
dk.seges              SEPTORIAHU                                        Septoria Humidity Model   
                      CPO                                         CPO model (diseases and pest)   

                                                                                          pests  \
no.nibio.vips         PSILARTEMP                                                       [PSILRO]   
                      DELIARADIC                                                       [HYLERA]   
                      MAMESTRABR                                                       [BARABR]   
                      PSILAROBSE                                                       [PSILRO]   
                      DELIARFOBS                                               [HYLERA, HYLEFL]   
                      NAERSTADMO                                                       [PHYTIN]   
                      ALTERNARIA                                                       [ALTESO]   
                      NEGPROGMOD                                                       [PHYTIN]   
                      SEPAPIICOL                                                       [SEPTAP]   
                      BREMIALAC

In [ ]:
hub.list_dss(View)


## Access to one DSS

In [ ]:
#PSI=hub.get_dss(ModelId='no.nibio.vips', DSSId='PSILARTEMP') 
# hub[dss][model]
PSI=hub.get_model(dss="no.nibio.vips", model='PSILARTEMP')
print(PSI)# PSILARTEMP

In [ ]:
# 
weather_data = ...
PSI(weather_data)
PSI.input_DSS_weather_model_json(TimeStart="2021-01-01", 
                                 TimeEnd="2021-12-31",
                                 weatherDataService='Finnish Meteorological Institute measured data',
                                 parameters=[1002],
                                 stationId=[101104])

In [ ]:
ds=PSI.get_data_model(modelInput='model_input_weatherdata.json')
ds

In [ ]:
ds.to_dataframe().plot.line()

In [ ]:
PSILAROBS= hub.get_dss(DSSId='PSILAROBSE')


In [ ]:
PSILAROBS.input_DSS_fieldobservation_model_json(fieldobservation=r"C:\Users\mlabadie\Documents\GitHub\dss\src\openalea\dss\fieldobservation.json")

In [ ]:
PSILAROBS.get_data_model(modelInput="model_input_fieldobservation.json")